In [1]:
import argparse
import gym
from gym import wrappers
import os.path as osp
import random
import itertools
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import os

In [2]:
#os.chdir("atari")
from run_dqn_atari import *
import dqn
from dqn_utils import *
from atari_wrappers import *
import pickle

In [4]:
# Get Atari games.
benchmark = gym.benchmark_spec('Atari40M')
# Change the index to select a different game.
task = benchmark.tasks[3]#Pong
# task = benchmark.tasks[1]#breakout
# task = benchmark.tasks[6]#spaceinvader
# task = benchmark.tasks[4]#Qbert
# Run training
seed = 0 # Use a seed of zero (you may want to randomize the seed!)
env = get_env(task, seed)
sess = get_session()
num_timesteps=task.max_timesteps
   

AVAILABLE GPUS:  ['device: 0, name: Tesla K80, pci bus id: 0000:00:04.0']


In [5]:
def stopping_criterion(env, t):
    # notice that here t is the number of steps of the wrapped env,
    # which is different from the number of steps in the underlying env
    return get_wrapper_by_name(env, "Monitor").get_total_steps() >= num_timesteps

In [6]:
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('pong.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
graph = tf.get_default_graph()
#q_func=atari_model

In [7]:
# q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_func')
# target_q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_q_func')

In [9]:
sess.graph.get_operations()
q_func_val= graph.get_tensor_by_name('target_q_func/action_value/fully_connected_1/BiasAdd:0')

In [10]:
img_Input=graph.get_tensor_by_name("Placeholder:0")
img_Input1=graph.get_tensor_by_name("Placeholder_1:0")
img_Input2=graph.get_tensor_by_name("Placeholder_2:0")
img_Input3=graph.get_tensor_by_name("Placeholder_3:0")
img_Input4=graph.get_tensor_by_name("Placeholder_4:0")

In [11]:
model_initialized = False
num_param_updates = 0
mean_episode_reward      = -float('nan')
best_mean_episode_reward = -float('inf')
last_obs = env.reset()
LOG_EVERY_N_STEPS = 1000

rewards_record = []
mean_episode_reward_record = []
best_mean_episode_reward_record = []

replay_buffer_size=2
frame_history_len=4
replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len)

In [ ]:
for t in itertools.count():
    if stopping_criterion is not None and stopping_criterion(env, t):
        break 
         
    if t == 0:
        # initializing the buffer
        ##step 1
        idx = replay_buffer.store_frame(last_obs)  
        act, reward, done = env.action_space.sample(), 0, False
        last_obs, reward, done, info = env.step(act)
        replay_buffer.store_effect(idx, act, reward, done)

      
    idx = replay_buffer.store_frame(last_obs) 
    
    if idx==0:
        id_current,id_next=1,0
    else:
        id_current,id_next=0,1
    
    obs_batch      = replay_buffer._encode_observation(id_current)[None]
    act_batch      = replay_buffer.action[[id_current]]
    rew_batch      = replay_buffer.reward[[id_current]]
    next_obs_batch = replay_buffer._encode_observation(id_next)[None]
    done_mask      = np.array([1.0 if replay_buffer.done[idx] else 0.0], dtype=np.float32)
    
    q_vals=sess.run(q_func_val,{img_Input:obs_batch,img_Input1:act_batch,img_Input2:rew_batch,img_Input3:next_obs_batch,img_Input4:done_mask})
    act = np.argmax(q_vals)
    last_obs, reward, done, info = env.step(act)
    replay_buffer.store_effect(idx, act, reward, done) 
    
    if done == True:
        last_obs = env.reset()
        done = False
    
    
    episode_rewards = get_wrapper_by_name(env, "Monitor").get_episode_rewards()
    if len(episode_rewards) > 0:
        rewards_record.append(episode_rewards[-1])

    if len(episode_rewards) > 0:
        mean_episode_reward = np.mean(episode_rewards[-100:])
        mean_episode_reward_record.append(mean_episode_reward)

    if len(episode_rewards) > 100:
        best_mean_episode_reward = max(best_mean_episode_reward, mean_episode_reward)
        best_mean_episode_reward_record.append(best_mean_episode_reward)
    
    if t % LOG_EVERY_N_STEPS == 0 and model_initialized:
        print("Timestep %d" % (t,))
        print("mean reward (100 episodes) %f" % mean_episode_reward)
        print("best mean reward %f" % best_mean_episode_reward)
        print("episodes %d" % len(episode_rewards))
        print("exploration %f" % exploration.value(t))
        print("learning_rate %f" % optimizer_spec.lr_schedule.value(t))
        sys.stdout.flush()
        record = ({"rewards":rewards_record, "mean_rewards": mean_episode_reward_record, "best_mean": best_mean_episode_reward_record})
        with open("./restore_rewards.pickle", "wb") as f:
            pickle.dump(record, f)
    